**First attempts**
Wednesday, February 26, 2020
2:03 PM
 
Got access to HPC via webCARS capacity clusters, simple ssh kevmoor@skybridge and kerb pass via git bash (which is just mingw).  May be good to switch to linux for local development.  Great that they are using slurm!  I may have also had to get access to matlab coder license via NILE.

Followed tutorial https://www.mathworks.com/videos/automatically-converting-matlab-code-to-c-code-96483.html compiled output on skybridge using:  cc main.cpp test_function.cpp test_function_terminate.cpp -o pleasework.  Would obviously use cmake for better compiling of more complex programs.  Didn't use srun to get slurm to give me actual compute time a resources.  Would need to do that, or sbatch for a auto batch run if we were running anything for real/substantial.

So, the matlab coder utility can output both source and executables, but were compiler issues right out of the box, which in my experience is typical for windows systems.  The code output for the simple function worked well enough for c and c++ output.  I turned off all of the overhead type things to make to code simpler and faster.

Now switching to the application compiler: I had issues trying to get it to create an installer with the runtime included.  I installed the runtime from NILE, but after a half hour of trying to get it to connect up, I stopped.  I'm sure it can be done, just isn't obvious to me.  The web installer works just fine and does output prebuilt executables as well, which run, but there is a lot of overhead time for it to start.  My simple function just output a few things to the terminal, so I couldn't just double click the executable, I had to run it from the terminal to see the output.

Now trying to run the version of OWENS I have through these tools:  Using the code generation readiness utility (right click a function and at the bottom), it said that the following were unsupported: fscanf, ismembc, str2num, warning, readtable. When I fixed those, and the syntax errors, it didn't like variable length arrays and made me pre-allocate them.

Ok, since I don't have a driver for OWENS with the proper input files yet, I switched to Brandon's blade optimization.  It had 889 unsupported Function calls, from regexp, to cd, to sscanf, to warning, to clc, and lots and lots of others.  So, I switched to the application compiler and it did compile (different than the C++ compilation, the application compiler requires the matlab runtime overhead).  But… it didn't run, error message says that Matlab compiler doesn't support modifying the search path, and I'd expect other errors would come up after that is fixed.  So, it's going to take a non-trivial amount of work to get this code ready for the matlab converter.  Likely can be done in-stride with the other development, but will need to be deliberate from the start.  If we have <10 auto check issues in OWENS now, but over 800 in the other optimization code, it sounds like we should definitely start going in the right direction NOW.  

Also, during my reading, it appears that matlab can compile binaries for other operating systems, but that it needs to be compiled from the target system (i.e. run the matlab compiler on a mac for a mac binary).  Another interesting thing was that the compiled matlab (according to the tutorials) isn't necessarily going to be faster than the interpreted matlab.  The c code may be faster, but may be due in part to the changes required to get matlab to spit out the code.
 
 
**Fast Input Files First Look**
Thursday, February 27, 2020
11:21 AM
 
Downloaded the example FAST input files from the docs, seems straightforward, similar to what I've used before with CCBlade.  Looking through the other files, I had the following thoughts:
 
We will need to choose a frame of reference for the blade geometry relative to the existing one: 
A) Extreme precone/curve and have to resolve the fact that the tower will be 90 degrees to the turbine without a transformation
B) Negative precone/curve and 90 degree transformation on the hubs.  
C) Use separate definitions rather than trying to use the old with modifications
 
The material properties, airfoil data, and initialization/driver files look to be straightforward to comply with for inputs to OWENS.
 
 
**Running Owens**
Monday, March 2, 2020
10:19 AM
Pulled the OffshoreVAWTmodels - Copy.zip file off of \\snl\Collaborative\windandwater\4BrandonE which appears to be a snapshot of the models.  Following the readme and running the transient driver worked, but there was an issue with wavec in plotting.  Turning off the hydro analysis and it seems to be running fine now.  Hasn't completed after a few minutes.  
 
**Write Deployable MATLAB Code**
Tuesday, March 3, 2020
3:56 PM
 
https://www.mathworks.com/help/compiler_sdk/ml_code/writing-deployable-matlab-code.html
 
We should start with only the code we want to deploy on unix: i.e. pre/post processing may be able to be postponed

 




**Compiling: Resolving Unsupported Function Calls**
Monday, March 2, 2020
10:30 AM
Adding paths: the compiler will figure it out on its own, but matlab won't, so add path if not deployed.
if (~isdeployed)
    VAWT_Toolbox_path_main = '../deps/OWENSvawtFoa/';
    addpath(VAWT_Toolbox_path_main)
end
Here are a list of supported functions: https://www.mathworks.com/help/simulink/ug/functions-and-objects-supported-for-cc-code-generation.html
 
Running the matlab code readiness tool on platformTests_transient_Mkprop, yielded the following unsupported functions.  The following are my work-arounds/new functions.
 
Fscanf - several other options in the list of supported functions, like fseek
 
Addpath - directly call out all paths, keep them in structures to be clean.
 
Str2num - use str2double
 
Cd - restructure code, see: https://www.mathworks.com/help/compiler_sdk/ml_code/writing-deployable-matlab-code.html
 
Robust way of extracting the file directory regardless of / or \:
last_delimiter = find(or(inputfile == '/', inputfile == '\'));
fdirectory = inputfile(1:last_delimiter(end));
 
 
Ismembc - use ismember
 
Warning - convert to print statement
 
Importdata - use an approved file reading method from link, may have to write small function to get the right format
 
Pwd - same as cd
 
Keyboard - restructure code: should not be hitting the keyboard in the middle of a batch parallel run on a HPC cluster
 
Strsplit - write own splitter logic, or use something like strfind in the approved list
 
Movefile - restructure so moving is not needed, or read in and rewrite the file elsewhere, and then delete the original file.
 
Getframe - This has to do with creating a video.  It is possible to restructure the code to simply output images and post process them into a video.  Also may be problematic to include image creation that uses gpu if there is no gpu on the cluster being run on, so may need to push to post processing, depends on if gpu is used etc.
 
Genpath - same as cd
 
VideoWriter - same as getframe
 
Textscan - same as fscanf
 
Movegui - same as getframe
 
Sscanf - same as fscanf
 
Clc - remove as it is matlab specific to the console.  Could put in printing levels.
 
Unix - remove since this is specific to wavec, seems to be dependent on hard coded file paths, and could easily be unified for the coupling with the new hydro tool.
 
Eigs - use one of the other eigenvalue/vector functions from the link
 
Writetable - use something like fwrite or fprintf
 
Cell2mat - restructure to not require this type of conversion
 
Annotation - move to post processing or include in legend, or remove.
 
 
All Files called by platformTests_transient_Mkprop 
 
https://www.mathworks.com/help/matlab/matlab_prog/identify-dependencies.html





 
**Successfully Compile OWENS**
Tuesday, March 3, 2020
5:17 PM
 
Put all required code in single repository and begin tracking changes
Get it running again in that environment
Create Short Unit Test to ensure code changes don't break physics (check #s based on original code)
Implement changes required as previously recorded
Attempt to create code and compile, also attempt to compile directly
Resolve any final issues associated with compilation failure
Successfully compile
Run on HPC/Unix
Resolve issues associated with running
Potentially merge changes with Brandon's changes
Map out next steps - DAKOTA and Parallelization, interfacing with other packages, cleaning up/simplifying code
 

How to preinitialize arrays of structs:

https://www.mathworks.com/help/simulink/ug/defining-arrays-of-structures-for-code-generation.html

It looks like you just have to initialize the correct type of variable, not necessarily the size, ie use zeros if you don't know the NxM etc.  

Some best practices
https://www.mathworks.com/help/simulink/ug/best-practices-for-defining-variables-for-c-c-code-generation.html

_______________________

If the type is unchanging and easily inferred (i.e. not a for loop), then you don't have to declare it.  

All variables used must be declared on all execution paths (i.e. must always have an else for each if, or be declared outside the if)

**All if statements must have an else, or the equivalent such that all --LOCAL-- execution paths possible are covered**

Structs must be build in the same order on all execution paths and cannot be used until they are finished being constructed

_____________________________

**Mar 24 2020**
4:55pm

Finally got it to create the source code, 91 commits later, each with some sort of restructuring/typsetting.  About 30% was done with the regular matlab editor suggestions, another 70% with the coder GUI in matlab, and the last 10% post compilation via mex (built into the coder GUI).  The most difficult issues to figure out were also in the 70% chunck.  Basically, you have to code as if you were coding in C++: declare your variables, and don't change their types or sizes after they are declared.  You can have variable size arrays, variable at the time of initialization.  I haven't played with it, but it appears that even that can be shut off for speed in the coder settings.  It is really frustrating to do this in matlab since whenever you make a mistake (like accidently creating a row vector instead of a column vector), it just plows through the rest of the program and resizes/retypes everything without telling you.  For this reason, it is my opinion that matlab is a terrible development tool. It incentivizes terrible coding practice, with and emphasis on matrix based syntax, opposite to any HPC code.  It is 2x slower with code optimized for matlab, then 1000x slower for code written to satisfy the coder requirements, also further incentivizing the wrong direction.  So you have to essentially write obtrusive matlab code, making up tons of sub functions to try to recapture the 60% of capability you loose in conforming with the coder requirements, then you have to generate the C++ code, and then compile it.  Why not just start with julia, retain 100% of your functionality, and have it run on HPC resources out of the box, and optimize your code as needed?  Good luck optimizing the C++ output of matlab.

This restructuring took approximately 2 weeks of work, for which I am confident I could have just translated OWENS into Julia. 

_________________

**Mar 25** 1:00pm

Speed comparison of compiled code:

- Matlab: 17s
- Matlab Mex: 34s
- Compiled C: 61s

In addition to yesterday's log, I'd say that compiling matlab is a solid loser. 92 Commits, 4 weeks, 228k Additions, and 112k Deletions to get a 3x slowdown.  If that effort were put towards translating into Julia, it would have likely taken less effort, and given a likely 2x or more speedup.  Why less effort?  Julia syntax is very similar to matlab, and julia will warn you before bulldozing through the code redefining all of your types and sizes.  For the VAWT AC code, it took me less than 8 hours to translate the code, then 2x that again to find the handful of array orientation mistakes made during the translation (since matlab 1D arrays are typically column vectors instead of row vectors, and zeros(5) makes a 5x5 instead of a straight 5 array of zeros, etc).  The owens repository I got to run C code is 12110 lines of code long. The C output is 29375 lines, 3642 for the .h and 25733 for the .c  VAWT-AC julia is 882 lines, with 1753 for the compiled root finding dylib, the matlab is 1046 lines.

Yes, the matlab code could be written better to get the c code more optimized, but wouldn't it be time better spent (and a heck of a lot easier) going stright for a HPC code like Julia.  That way we wouldn't have to go through three stages to get the code compiled.

Used the example main.c and main.h, and the following command to compile (using the packaged output of coder): gcc -Wall *.c -o pleasework.  The -Wall shows extra warnings the compiler normally ignores

**Mar 25** 5:30pm

Code review with Brandon where we agreed to

- Continue on the matlab path until we have the big pieces in place, then evaluate as to the best path for HPC
- Try the full cactus sim time and see time difference
- Try turning off the file save portion and see time difference

- Most immediate next step is coupling method
- Then Translating the modal and flutter analysis
- Then unsteady stall
- Also getting rally setup




||Matlab|Matlab MEX|Compiled C|Octave|
|---|---|---|---|---|
|Lines of Code|12,110 lines |N/A|29,375 lines|Runs .m file|
|Execution Time Short (0.1s) & File Output|21s|34s|61s|~400s|
|Execution Time Short (0.1s), No File Output|14s|19s|60s|~400s|
|Execution Time LONG (19.1s), No File Output|13m 40s(9s /100 time steps)|4m 31s (3s/100 time steps)|~3hr (2min/100 time steps)|~15hr (10min/100 time steps)|


**March 31 2020** 11:45am

Profiling the super slow C code:

brew install gperftools
brew install graphviz
brew install ghostscript

I copied the input and output folders into my coder "package" unzipped folder with the example main.cpp and main.h from the codegen folder copied. I did also switch to C++, but is seems to run in about the same time as C 

I had to disable address space layout randomization (ASLR) (https://stackoverflow.com/questions/10562280/line-number-in-google-perftools-cpu-profiler-on-macosx)

so in the end I ran the following commands

    c++ -std=c++11 -lprofiler -g -Wl,-no_pie *.cpp -o please4work

    env CPUPROFILE=/tmp/prof.out ./please4work 

    pprof --web --functions ./please4work /tmp/prof.out

(Note that you have to change the default .svg program in finder.  I changed mine to safari)

**Profiler Results**

So, it looks like there is some serious time being spent in the StructuralDynamicsTransient function, and not really the sub functions.  Looking at the code output, it looks like this is likely a memory issue where matlab is very inefficiently forcing c++ to duplicate its memory management schemes.  I'm sure a CS major could tell us the clear problem and solution in about 60 seconds.

![Profiler Results](./figs/profiler_output.png)

**April 6 2020** 12:20pm

Working on debugging the slowdown.  Also got the modal analysis to compile, but is not outputting the correct answers, possibly a complex number issue.  Ways I've tried to attack this issue

 - Compiling binary directly from matlab: Issues in getting the compiler to link to matlab.  Issue is common, but poorly documented

 - Running mex from octave: Requires some significant intrusive changes: https://hgomersall.wordpress.com/2013/02/19/squishing-matlab-mex-files-into-octave/
 - Running mex from python: There is a matlab API for this
 - Creating mex file from octave: spent about 15 min on this and it did compile, but will not run.
 
However... The mex function calls the matlab libraries, so the mex itself doesn't contain all the code.  Therefore, the option would be to compile the matlab code (possibly with the mex as part of it), but that requires the matlab runtime.
 
Now on to the other ways I've tried to attack this issue:
 
 - C++ debugger: gdb.  Some non-trivial issues getting it to run - keychain permissions, etc.  Turns out none of them work and you just have to run it as sudo.
 
**April 6 2020** 3:50pm

Ok, using the gdb and the simple debugging principle that if the is one bottle neck taking a large amount of time, then there is a high probability randomly debugging and stopping the debugging will land you at the trouble spot.  Doing the process several dozen times, yielded the following, all in structuralDynamicsTransient.cpp (keep in mind, it could have landed anywhere in the program):


|C++ Line|# Hits|
|---|---|
|473|15|
|429|5|
|428|3|
|471|1|
|427|1|
|472|1|

They all map to a **SINGLE** matlab line with basic matrix multiplication of fixed size matrices (for a given program run):
structuralDynamicsTransient.m:339

```matlab
Kg = transMatrix'Kg*transMatrix;
```

why on earth it spits out the following c++ code is a mystery yet to be solved

```c++
  i = a->size[0] * a->size[1];
  a->size[0] = model_jointTransform->size[1];
  a->size[1] = model_jointTransform->size[0];
  emxEnsureCapacity_real_T(a, i);
  loop_ub = model_jointTransform->size[0];
  for (i = 0; i < loop_ub; i++) {
    m = model_jointTransform->size[1];
    for (boffset = 0; boffset < m; boffset++) {
      a->data[boffset + a->size[0] * i] = model_jointTransform->data[i +
        model_jointTransform->size[0] * boffset];
    }
  }

  if ((a->size[1] == 1) || (Kg->size[0] == 1)) {
    i = y->size[0] * y->size[1];
    y->size[0] = a->size[0];
    y->size[1] = Kg->size[1];
    emxEnsureCapacity_real_T(y, i);
    loop_ub = a->size[0];
    for (i = 0; i < loop_ub; i++) {
      m = Kg->size[1];
      for (boffset = 0; boffset < m; boffset++) {
        y->data[i + y->size[0] * boffset] = 0.0;
        inner = a->size[1];
        for (coffset = 0; coffset < inner; coffset++) {
          y->data[i + y->size[0] * boffset] += a->data[i + a->size[0] * coffset]
            * Kg->data[coffset + Kg->size[0] * boffset];
        }
      }
    }
  } else {
    m = a->size[0];
    inner = a->size[1];
    loop_ub = Kg->size[1];
    i = y->size[0] * y->size[1];
    y->size[0] = a->size[0];
    y->size[1] = Kg->size[1];
    emxEnsureCapacity_real_T(y, i);
    for (j = 0; j < loop_ub; j++) {
      coffset = j * m;
      boffset = j * inner;
      for (b_i = 0; b_i < m; b_i++) {
        y->data[coffset + b_i] = 0.0;
      }

      for (k = 0; k < inner; k++) {
        aoffset = k * m;
        totalNumDOF = Kg->data[boffset + k];
        for (b_i = 0; b_i < m; b_i++) {
          i = coffset + b_i;
          y->data[i] += totalNumDOF * a->data[aoffset + b_i];
        }
      }
    }
  }

  if ((y->size[1] == 1) || (model_jointTransform->size[0] == 1)) {
    i = Kg->size[0] * Kg->size[1];
    Kg->size[0] = y->size[0];
    Kg->size[1] = model_jointTransform->size[1];
    emxEnsureCapacity_real_T(Kg, i);
    loop_ub = y->size[0];
    for (i = 0; i < loop_ub; i++) {
      m = model_jointTransform->size[1];
      for (boffset = 0; boffset < m; boffset++) {
        Kg->data[i + Kg->size[0] * boffset] = 0.0;
        inner = y->size[1];
        for (coffset = 0; coffset < inner; coffset++) {
          Kg->data[i + Kg->size[0] * boffset] += y->data[i + y->size[0] *
            coffset] * model_jointTransform->data[coffset +
            model_jointTransform->size[0] * boffset];
        }
      }
    }
  } else {
    m = y->size[0];
    inner = y->size[1];
    loop_ub = model_jointTransform->size[1];
    i = Kg->size[0] * Kg->size[1];
    Kg->size[0] = y->size[0];
    Kg->size[1] = model_jointTransform->size[1];
    emxEnsureCapacity_real_T(Kg, i);
    for (j = 0; j < loop_ub; j++) {
      coffset = j * m;
      boffset = j * inner;
      for (b_i = 0; b_i < m; b_i++) {
        Kg->data[coffset + b_i] = 0.0;
      }

      for (k = 0; k < inner; k++) {
        aoffset = k * m;
        totalNumDOF = model_jointTransform->data[boffset + k];
        for (b_i = 0; b_i < m; b_i++) {
          i = coffset + b_i;
          Kg->data[i] += totalNumDOF * y->data[aoffset + b_i];
        }
      }
    }
  }
```

Trying the following shaves 2seconds/ 100 time steps off.  Perhaps it will give a better c++ output

```matlab
% Kg = transMatrix'*(Kg*transMatrix);
Kg = full(sparse(transMatrix')*sparse(Kg)*sparse(transMatrix));
```

Much much better! 24X Faster!  Matlab tests pass and c++ test passes for transient!

||Matlab|Matlab MEX|Compiled C++|Octave|
|---|---|---|---|---|
|Execution Time Short (0.1s) & File Output|21s|NA|5s|many minutes|
|Per 100 Steps|7s|2s|5s|many minutes|


This is the output code, (that one liner was in its own function to start with, and now matlab finally puts this in its own function) and the code is significantly simpler.  Also, as a note, it didn't seem to have any effect when I tried to initialize variables outside of a for loop, it seemed to just do as it pleased.

```c++


static void applyConstraints(emxArray_real_T *Kg, const emxArray_real_T
  *transMatrix)
{
  emxArray_real_T *b_transMatrix;
  int i;
  int loop_ub;
  emxArray_real_T *this_d;
  int cend;
  emxArray_int32_T *this_colidx;
  int t3_m;
  emxArray_int32_T *this_rowidx;
  emxArray_real_T *t2_d;
  emxArray_int32_T *t2_colidx;
  emxArray_int32_T *t2_rowidx;
  emxArray_real_T *t3_d;
  emxArray_int32_T *t3_colidx;
  emxArray_int32_T *t3_rowidx;
  int this_m;
  int this_n;
  emxInit_real_T(&b_transMatrix, 2);
  i = b_transMatrix->size[0] * b_transMatrix->size[1];
  b_transMatrix->size[0] = transMatrix->size[1];
  b_transMatrix->size[1] = transMatrix->size[0];
  emxEnsureCapacity_real_T(b_transMatrix, i);
  loop_ub = transMatrix->size[0];
  for (i = 0; i < loop_ub; i++) {
    cend = transMatrix->size[1];
    for (t3_m = 0; t3_m < cend; t3_m++) {
      b_transMatrix->data[t3_m + b_transMatrix->size[0] * i] = transMatrix->
        data[i + transMatrix->size[0] * t3_m];
    }
  }

  emxInit_real_T(&this_d, 1);
  emxInit_int32_T(&this_colidx, 1);
  emxInit_int32_T(&this_rowidx, 1);
  emxInit_real_T(&t2_d, 1);
  emxInit_int32_T(&t2_colidx, 1);
  emxInit_int32_T(&t2_rowidx, 1);
  emxInit_real_T(&t3_d, 1);
  emxInit_int32_T(&t3_colidx, 1);
  emxInit_int32_T(&t3_rowidx, 1);
  b_sparse(b_transMatrix, t2_d, t2_colidx, t2_rowidx, &cend, &loop_ub);
  b_sparse(Kg, this_d, this_colidx, this_rowidx, &this_m, &this_n);
  d_sparse_mtimes(t2_d, t2_colidx, t2_rowidx, cend, this_d, this_colidx,
                  this_rowidx, this_n, t3_d, t3_colidx, t3_rowidx, &t3_m,
                  &loop_ub);
  b_sparse(transMatrix, t2_d, t2_colidx, t2_rowidx, &cend, &loop_ub);
  d_sparse_mtimes(t3_d, t3_colidx, t3_rowidx, t3_m, t2_d, t2_colidx, t2_rowidx,
                  loop_ub, this_d, this_colidx, this_rowidx, &this_m, &this_n);
  i = Kg->size[0] * Kg->size[1];
  Kg->size[0] = this_m;
  Kg->size[1] = this_n;
  emxEnsureCapacity_real_T(Kg, i);
  emxFree_real_T(&b_transMatrix);
  emxFree_int32_T(&t3_rowidx);
  emxFree_int32_T(&t3_colidx);
  emxFree_real_T(&t3_d);
  emxFree_int32_T(&t2_rowidx);
  emxFree_int32_T(&t2_colidx);
  emxFree_real_T(&t2_d);
  for (i = 0; i < this_n; i++) {
    for (t3_m = 0; t3_m < this_m; t3_m++) {
      Kg->data[t3_m + Kg->size[0] * i] = 0.0;
    }
  }

  for (loop_ub = 0; loop_ub < this_n; loop_ub++) {
    cend = this_colidx->data[loop_ub + 1] - 1;
    i = this_colidx->data[loop_ub];
    for (t3_m = i; t3_m <= cend; t3_m++) {
      Kg->data[(this_rowidx->data[t3_m - 1] + Kg->size[0] * loop_ub) - 1] =
        this_d->data[t3_m - 1];
    }
  }

  emxFree_int32_T(&this_rowidx);
  emxFree_int32_T(&this_colidx);
  emxFree_real_T(&this_d);
}

```

**April 6 2020** 5:40pm

Moving on to the modal analysis verification issue:

||Matlab|Matlab MEX|Compiled C++|Octave|
|---|---|---|---|---|
|Correct Output|yes|yes|no-visually is way off|yes-visually, appears octave writes -0.0 instead of 0.0|
|Execution Time|11s|not recorded|80s|~5min|

So, we do have **a** way to run it on the compute nodes via octave.  It's slow, but does work.

Ok, let's try the debugger route.

compile the code using the -g flag to include the debugger ability
- c++ -std=c++11 -g *.cpp -o pleaseMwork

now run the code in the directory it was compiled from (i.e. copy the necessary run folders into the build directory) (I installed gdb via brew) (cheat sheet for gdb: https://darkdust.net/files/GDB%20Cheat%20Sheet.pdf)
- sudo gdb ./pleaseMwork

Place breakpoints and run (the first is so the program will start debugging right off)
- break main.cpp:61
- break extractFreqDamp.cpp:182
- run



**April 7 2020** 8:30am

It catches at a break point and then we can continue (c) or quit by (cntrl-c). (info locals) shows all of the local variables.  (print variablename) prints the variable name.  Keep in mind, c++ matrices are row arrays that loop around, so to get the second row, column one of a 414x414 array you'd access 414 (remember zero indexing), and the command would likely be "print variable->data[414]".  

I've identified that its the damping matrix, C, that has all the values negated.  I've traced it backwards to the timoshenkoelement 

**April 7 2020** 3:30pm

it came down to this line of code calcTimoNL.m at about line 580:

```matlab
[Ce] = mapMatrixNonSym([zm,C12,C13,C14,zm,zm;
     -C12',zm,C23,C24,C25,C26;
     -C13',-C23',C33,C34,C35,C36;
     -C14',-C24',C43,C44,C45,C46;
     zm,-C25',-C35',-C45',zm,zm;
     zm,-C26',-C36',-C46',zm,zm]);

```
All of the inputs had the correct signs, but the matrix it created did not.  I explicitely built the matrix element by element and it changed the C++ output from a very odd mapping scheme to nearly following what I implemented (which was much simpler and easier to read).  Still isn't quite right, but at least now the mode shape numbers are just negated, but appear to have the correct magnitude of values.  Will debug the C,K,M matrices again.

4:00pm

Ok, it looks like there is a bug in the translator, it is mixing up the rows.  While true that row 3 index (3,1) (i.e linear index 24 is set to the negative C12 value accessed, the matlab code is assigning it to the wrong index, like it is getting confused trying to write more "efficient" code, which is only ending up creating terrible non-working code.

```c++
  //  Row 1
  ktemp2_tmp = elStorage->C12[0] * Oel[2];
  ktemp2[24] = ktemp2_tmp;
    .
    .
    .
  //  Row 3
  ktemp2[2] = -ktemp2_tmp;
```

Now that I have had a first in-depth scrape with matlab's C++ code output, I'd give it a D-.  In other words, good effort, but the product is scrappy at best for a HPC quality code. It is difficult to follow (without the master Matlab script side by side it would be impossible), it has redudnant code everywhere, including two and three copies of the same functions all side by side (in a 500 line matlab script, that means that each function is ~1500 lines for a total of about 5000 lines, and the other functions aren't called), and finally, it fails at what it is supposed to be best at.  Matlab stands for matrix laboratory, but when it spits out a complex, non-standard matrix multiply function at random that is 24-50x slower than the standard, while using the standard fast method elsewhere.

4:40pm

It appears that the translator makes the same assumption I would when creating a matrix, that to access the 3,1 element of a 12x12 matrix, one would specify 24, assuming row folding.  However that is not the case.  It uses column folding, so element 24 accesses 1,3, or the transpose.  Since the matrix at hand is negative symmetric, we had the masked issue.

5:40pm

Both are using row major and both match, its just that the debugger makes it look column major, and uses the array syntax that matlab would use for column major (i.e. commas).  So matlab didn't get this part wrong, I misunderstood the debugger's/c++, but the rest of the auto generated code is still scrappy.  This means I need to backtrack to the function that outputs the mode shapes... :(

As a note, this process is quite tedious:

Change matlab code -> Generate c++ code (~3min) -> setup folder structure (30s) -> compile c++ (30s) -> get debugger running (2min) -> step to desired location and print (1min).  That's 7 minutes of overhead for each code change.  Using Julia it would be about 20 seconds, and matlab it's like 10 seconds thanks to the multimillion dollar debugger.

Ok, we know that the mode shapes are off, let's check the Mg, Cg, Kg matrices.  
- A half dozen samples of the M matrix all over show equivalence (M: linearAnalysis:145, C: linearAnalysis:415
- Same story for the K matrix (similar location)
- Same story for C matrix (similar location)

**April 8 2020** 8:35am

- Mg_Total good
- Cg_Total good
- Kg_Total good

Dive into eigsolve again.

- sysMat appears to be the same

4:30pm

Looking into all of the different ways to solve the eigenvalue problem: eigs(A,B,nmodes,sortingmethod), polyeig(M,C,K), eig(A), eig(A,B), etc.  It appears that while the eigs implementation in the code appeared to give different results, it is now appearing that that was only since it was sorted from smallest to largest (absolute, including imaginary number).

However, using the sorted eig output, the mode shapes are still different from the sorted eigs output, even when using eigs to solve for all of the modes.  It does appear however that the worst residual of the eig output is much lower than eigs (e-07 as opposed to e-02).  Therefore it may be that the mode shapes from eig are more accurate than eigs???  This isn't the case for the dominant modes though, since the eigs residual is much lower (eigs 1e-16 and eig 1e-10).  So the differing methods don't match, and then when we compile the code and run it, there is some similarity in the numbers, but some of them are still way off.  So, I vote that since I've spent over 2 days on this single particular problem, and it appears to be rooted in the eigen solve, I move on and leave these thoughts for when we come back to it.  However, my official opinion is that we should just switch to a more scientific computing friendly environment, and ditch trying the get the Grinch to do what we want with a 32.5 foot pole.

    1) Need to develop a real validation case
    2) If none of the eig solving techniques work in C++ (i.e. we can't use them for our purposes), we decide to either
        a) Just use octave and don't worry about the 5 min run time
        b) Develop our own eigensolve algorithm that plays nicely with the C++ translator



**April 9 2020** 11:59am

Moved on to the flutter analysis.  It looks like the manual flutter is already implemented alongside the modal analysis.  Working on the automated flutter analysis.  For a coarse number of modes and RPM levels it is having a terrible time converging.  I am going to try a root solver instead of damped iteration.  

Switched to a 1D minimizer since it doesn't appear that some mode/RPM levels have solutions.  Another observation is that this auto flutter method is showing that the underlying analysis, physically accurate or not, can be quite discontinuous at the flutter solutions.

**April 13 2020** 4:30pm 

Compiled and successfully ran the automated flutter analysis.  No verification yet since that depends on the eigensolve.

Found out about and created a space on confluence.sandia.gov.  They have an integrated instance of lucidchart!  I don't know if that includes the realtime collaboration in the lucidchart instance itself, but it's great!  Also appears to have general collaborative documents -> i.e. could be a replacement for google docs.

Using the sandia instance of lucidchart, I created the following flowcharts. (Can also export as .vdx for legacy compatability)

![Internal Code Structure](./figs/OWENS.pdf)
![High Level IO Draft](./figs/ARCUS_IO.pdf)


6:15 pm

Plotting the modes shows only minor discrepancies (discounting inconsistencies between phases and directions) between the matlab and C++, except for mode 5, which one of the phases doesn't match at all. Left to right:

EIGS Output ...............................................................  EIG Output ...............................................................  C++ Output (Based on EIG code)

![Mode 5](./figs/mode5.png)

**April 14 2020** 7:00pm

Today I learned that eigenvector magnitude is meaningless, that the vector magnitude is essentially a function of good numerical methods.  Therefore, the unit test needs to be updated to compare the normalized mode shapes.  Now as far as switching up the phases and directions, the back of my mind is telling me that is not a problem also because of the numerical methods, mostly for higher order modes.  I also learned that the polyeigs output versus the eigs square output are the same modeshapes, it is just that the square matrix with the repeated diag and zeros output repeats the modes.  It may be faster to use the polyeigs function due to this.  Will need to look into that tomorrow after updating the unit tests, and also putting the results of the hand test here.

**April 15 2020** 10:55am

Figures as promised.  I also updated the modal verification method as planned and it all passes, even mode 5 which is very interesting, the scaling of the C++ of mode 5 must just have been very different than the others.  Would be interesting to plot the mode shapes with equal scaling, and also verify that the code is correctly using them for displacements.

Non Unitized, with the second/repeated set of eigenvectors included (first mode from the by hand script)

X-axis is node number, y-axis is nondimensionalized magnitude
![non_unitized_eigvec](./figs/non_unitized_eigvec.jpg)

When unitized, they are all exactly the same shape.
![unitized_eigvec](./figs/unitized_eigvec.jpg)

**April 16 2020** 10:15am

Timed results of differing eig methods, times are for the entire run:

|Method|Time|
|---|---|
|Polyeig|93s|
|Split out M EIG|94s|
|Single Matrix EIG|57s|
|Matlab single mat EIG|10s|

Also ran the profiler on the single matrix eig and the issue is within xzggev.cpp, which appears to be the eigensolve itself.  The function is basically illegible, so it seems that our 57 seconds is as good as we get without CS help and migrating the code permanently out of matlab.  It is potentially possible to play with the types/sizes (making them more concrete) going into the matlab solve to see if the output function can be simplified.

Hard coding the matrix size going into the eig solve gives a run time of 38 seconds (single matrix eig), so definitely some potential there.

**July 13 2020**

Switching to julia translation.  Aero module is put together, but some bugs in the dynamic stall and also in the RPI fundamental effects that I want to hold off on until my mind clears in a week (since I don't have a coding buddy).

Starting translation log: so that we know just how challenging and resource intensive it is to translate:

10:20am

file extension, comments, looked for auto translator, but don't think that it will work, plus it isn't outside eyes free...

' to "

fprintf to println

access elements ( to \[

";" not needed unless multiline-line

platformprop struct to declared mutable struct at top of file, should be changed to regular struct for speed and safety.

test_owens done: 10:50am, moving on to owens

include owens in previous file (should be later better organized)

change function header return to a return at the end of the file.

;

'

change varargin to the optional julia arguments with defaults

model mutable struct

fopen to open

translating find(or logic 11:20

11:45 switch 1:15pm, got it at 1:35

fgetl to readline

[

string2double to parse(Float64, string)

[stringa stringb] to string(stringa, stringb)

find to findall and associated logic 2:15
Multitasking during meeting and CACTUS questions
translating short readmesh function 3:50
outputs not in []
translating readBladeData
max to maximum for arrays
~ to ! for not
~ to _ for unused arguments/outputs

distractions with child... 10 min

Translating readBCdata.jl